In [ ]:
import pandas as pd
import re
pd.set_option('display.max_colwidth', -1)
import numpy as np

In [ ]:
df = pd.read_csv('phy.txt',delimiter='\001',header=None,encoding='UTF-8')

In [ ]:
df.columns = ['user_id', 'question_id','user_answer','right_answer',
'is_right',
'tag_code',
'description',
'difficulty',
'question_type',
'content',
'options',
'reason' ]
df.head(5)

In [ ]:
df_multiple = df.dropna()

In [ ]:
df_shorta=df[df['options'].isna()]

In [ ]:
df_multiple_wrong = df_multiple[df_multiple['is_right']==0]
df_shorta_wrong = df_shorta[df_shorta['is_right']==0]

In [ ]:
df_multiple_wrong.iloc[0]

In [ ]:
df_shorta_wrong.iloc[3][2]

In [ ]:
import ast
type(ast.literal_eval(df_shorta_wrong.iloc[3][3]))

In [ ]:
len(df_shorta_wrong)

In [ ]:
df_shorta_wrong.to_csv(path_or_buf='./short_ans_wrong.tsv',sep='\t')
df_multiple_wrong.to_csv(path_or_buf='./multiple_ch_wrong.tsv',sep='\t')

In [ ]:
kmap = pd.read_csv('phy_kmap.csv',encoding='UTF-8')
kmap.head(1)

In [ ]:
tag_ids = kmap[['tag_list']].iloc[0]
tag_ids[0].split(",")

In [ ]:
kmap_id_dict={}
kmap_code_dict={}
for row in kmap[['kmap_id','kmap_code','tag_list']].itertuples():
    if row[1] in kmap_id_dict:
        print('kid repeated ',row[1])
    elif row[2] in kmap_code_dict:
        print('k code repeated ',row[2])
    kmap_id_dict[row[1]]=row[-1].split(",")
    kmap_code_dict[row[2]]=row[-1].split(",")

In [ ]:
op_df = pd.read_csv('./sample.tsv',sep='\t',encoding='UTF-8')
op_df.head(5)

In [ ]:
def get_vecs():
    q_tags={}
    with open('./sample.tsv','r') as f:
        f.readline()
        for row in f.readlines():
            cols=row.split('\t')
            cols[-1]=cols[-1][:-2]
    #         print(cols[0])
    #         print(cols[1:])
            q_tags[cols[0]] = np.array(cols[1:],dtype=float)
    return q_tags

In [ ]:
def cluster_vecs_by_similarity(q_tags,min_similarity=0.85):
    def get_cosine_dist(vec1,vec2):
        return np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
    unq_tags=list(q_tags.keys())
    cluster_dict={}
    cluster_count=0
    clustered_q_tags=[]
    for k in q_tags:
        if k not in clustered_q_tags:
            clustered_q_tags.append(k)
            cluster_dict[cluster_count]=[k]
            for q,vec in q_tags.items():
                if q not in clustered_q_tags:
                    if get_cosine_dist(vec,q_tags[k])>=min_similarity:
                        clustered_q_tags.append(q)
                        cluster_dict[cluster_count]+=[q]
            cluster_count+=1
    return cluster_dict         

In [ ]:
clusters=cluster_vecs_by_similarity(q_tags,0.74)

In [ ]:

df_mul2=df_multiple_wrong[df_multiple_wrong['question_type']=='RADIO']
df_mul2.head()


In [ ]:
df_mul2.to_csv(path_or_buf='./mult_radio.tsv',sep='\t',encoding='UTF-8')

In [ ]:
df_reason=df_mul2[['reason']]

In [ ]:
res_lst=[]
for row in df_reason.itertuples():
    res_lst.append(re.sub('[\\u3010 \\u3011]','',row[-1]))

In [ ]:
df_mul2['reason2'] = pd.Series(res_lst)